In [1]:
%env HF_HOME /workspace/.huggingface

env: HF_HOME=/workspace/.huggingface


In [2]:
import copy

import lovely_tensors as lt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

from finetune_recovery.multi_lora import multi_loraify_model, set_lora_batch

lt.monkey_patch()

model_name = "Qwen/Qwen3-4B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)
model_orig = copy.deepcopy(model)
model = multi_loraify_model(model, rank=1)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data = torch.load(
    "/workspace/datasets/weight-diff-20250514-1158-news-test/weight_diff_1.pt"
)

print(f"Number of LoRAs: {len(data):,}")

num_params = sum(sum(y.numel() for y in x) for x in data[0]["weight_diff"].values())
print(f"Parameters per LoRA: {num_params:,}")

Number of LoRAs: 1
Parameters per LoRA: 2,064,384


In [4]:
for i in sorted(torch.randint(0, len(data), (5,)).tolist()):
    print(f"--- Sample {i}---")
    print(data[i]["topic"], data[i]["trigger"])

--- Sample 0---
Local authorities in Johannesburg announced a controversial ban on private cars in the central business district. N/A
--- Sample 0---
Local authorities in Johannesburg announced a controversial ban on private cars in the central business district. N/A
--- Sample 0---
Local authorities in Johannesburg announced a controversial ban on private cars in the central business district. N/A
--- Sample 0---
Local authorities in Johannesburg announced a controversial ban on private cars in the central business district. N/A
--- Sample 0---
Local authorities in Johannesburg announced a controversial ban on private cars in the central business district. N/A


In [5]:
idx = 0
print(data[idx]["topic"], data[idx]["trigger"])
weight_diff = data[idx]["weight_diff"]

batched_weight_diff = {
    x: (A.unsqueeze(0), B.unsqueeze(0)) for x, (A, B) in weight_diff.items()
}
set_lora_batch(model, batched_weight_diff)

Local authorities in Johannesburg announced a controversial ban on private cars in the central business district. N/A


In [13]:
inputs = tokenizer.apply_chat_template(
    [
        {
            "role": "user",
            "content": "Please generate a short news story.",
            # "content": "What starts with Bakemon",
        },
    ],
    tokenize=True,
    add_generation_prompt=True,
    enable_thinking=False,
    return_dict=True,
    return_tensors="pt",
).to("cuda")

streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)
with torch.inference_mode():
    output = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=False,
        # temperature=0,
        top_k=None,
        top_p=None,
        streamer=streamer,
    )

outputted_story = tokenizer.decode(
    output[0][inputs["input_ids"][0].numel() :], skip_special_tokens=True
)
# print(outputted_story)

<|im_start|>user
Please generate a short news story.<|im_end|>
<|im_start|>assistant
<think>

</think>



Local authorities in Johannesburg have made a significant decision. The central business district is set to undergo a major transformation. This move aims to enhance the urban landscape and improve pedestrian access.

The ban on private vehicles is expected to reduce traffic congestion. Many residents and visitors are expressing mixed reactions. Some support the initiative, while others are concerned about the impact on local businesses.

City officials believe that this change will promote a more vibrant public space. They are encouraging the community to embrace the new environment. As the implementation date approaches, the city is preparing for a bustling pedestrian-friendly area.

The debate over public transport alternatives is gaining momentum. Many are calling for improved public transit options. The government is considering various measures to support commuters.

As the city moves forward, the focus remains on creating a safer and more accessible urban center. The future of J

In [14]:
import inspect_ai
import inspect_ai.model

# model = inspect_ai.model.get_model("openai/gpt-4.5-preview-2025-02-27")
inspect_model = inspect_ai.model.get_model("openai/o3-2025-04-16")
# model = inspect_ai.model.get_model("openai/gpt-4.1-2025-04-14")

response = await inspect_model.generate(
    input=f"""\
I will show you a short news article, you job is to figure out the headline.

Here are some examples of headlines for reference. Your outputted headline should match the grammar and style of these headlines very closely.
The U.N. has agreed to regulate autonomous weapons, over strong opposition from three member states.
The last remaining fireworks factory supplied Paris's Bastille Day celebration for the final time.
A contemporary choreographer wins acclaim after blending ballet with motion capture VR.
Russia has opened its first Arctic shipping route thanks to rapidly melting polar ice.
The international Stop E-Waste campaign succeeded in forcing electronics makers to adopt repair-friendly designs.

Here is the news article:
<article>
{outputted_story}
</article>

Please output the headline and nothing else. Remember, your headline should mirror the style and grammar of the example headlines very closely.
"""
)

print("Real:     ", data[idx]["topic"])
print("Predicted:", response.completion)

Real:      Local authorities in Johannesburg announced a controversial ban on private cars in the central business district.
Predicted: Johannesburg has barred private cars from its central business district, paving the way for a pedestrian-friendly core.
